In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import numpy as np
sys.path.insert(0, "..")


In [ ]:
import trimesh
import matplotlib
import numpy as np
import meshplot as mp
import matplotlib.pyplot as plt

# Visualize the Implicit Field

In [ ]:
!ls -d pretrained/deform/*rej*/

In [ ]:
# log_path = "good_logs/deform/armidillo_back_stretch1e-1_bend1e-3_mine_nodetach_gclip10_lr1e-5_rejpresmp_5k_2021-May-25-04-55-14"
# log_path = "good_logs/deform/armidillo_front_stretch1e-1_bend1e-3_mine_nodetach_gclip10_lr1e-5_rejpresmp_2021-May-25-04-51-24/"
# log_path = "good_logs/deform/bar_twisting_stretch1e-1_bend1e-3_gclip10_lr1e-5_rejsmp_2021-May-25-13-58-02/"
# log_path = "good_logs/deform/cactus_side_stretch1e-1_bend1e-3_mine_nodetach_lr1e-5_7.5k_rejsmp_2021-May-25-13-26-44/"
# log_path = "good_logs/deform/cactus_y_rotate_stretch1e-1_bend1e-3_mine_nodetach_lr1e-5_7.5k_rejsmp_2021-May-25-13-27-20/"
# log_path = "good_logs/deform/cylinder_rotate_90d_stretch1e-1_bend1e-3_mine_nodetach_gclip10_lr1e-5_rejsmp_2021-May-25-13-59-09/"
# log_path = "good_logs/deform/cylinder_translate_stretch1e-1_bend1e-3_mine_nodetach_gclip10_lr1e-5_rejsmp_2021-May-25-13-59-57/"
# log_path = "good_logs/deform/dino_y-45d_stretch1e-1_bend1e-3_mine_nodetach_gclip10_lr1e-5_rejpresmp_2021-May-25-04-46-08/"
# log_path = "good_logs/deform/armidillo_side_stretch1e-1_bend1e-3_mine_nodetach_gclip10_lr1e-5_rejpresmp_5k_2021-May-26-00-07-48/"
# log_path = "good_logs/deform/cactus_translate_stretch1e-1_bend1e-3_mine_nodetach_lr1e-5_7.5k_rejsmp_2021-May-25-13-26-00/"
log_path = "good_logs/deform/cylinder_translate_stretch1e-1_bend1e-3_mine_nodetach_gclip10_lr1e-5_rejsmp_2021-May-25-22-56-05/"

In [ ]:
import yaml
import argparse
from train import dict2namespace
from pprint import pprint
from trainers.utils.utils import load_imf

# parse config file
with open(os.path.join(log_path, "config", "config.yaml"), 'r') as f:
    config = yaml.load(f)
cfg = config = dict2namespace(config)
cfg.save_dir = "logs"
os.makedirs("logs", exist_ok=True)
pprint(config)

epoch = None
trainer = load_imf(log_path, return_trainer=True, epoch=epoch)


# Load dataset

In [ ]:
import trimesh
import importlib
data_lib = importlib.import_module(cfg.data.type)
loader = data_lib.get_data_loaders(cfg.data, None)['train_loader']
for data in loader:
    break
    
print(data.keys())
handles = data['handles'].view(-1, 3).detach().cpu().numpy()
targets = data['targets'].view(-1, 3).detach().cpu().numpy()
print(handles.shape, targets.shape)

gtr_mesh = trimesh.Trimesh(
    vertices=data['gtr_verts'].cpu().view(-1, 3).numpy(),
    faces=data['gtr_faces'].cpu().view(-1, 3).numpy()
)
gtr_mesh.show()

# Get the output Mesh

In [ ]:
from trainers.utils.vis_utils import imf2mesh
from trainers.utils.igp_process import trimesh_to_o3dmesh
res = 256
out_mesh = imf2mesh(lambda x: trainer.decoder(x, None), res=res, normalize=True, bound=1.2)
out_mesh.show()


# Output information for drawing

In [ ]:
res_path = os.path.join(log_path, "results")
os.makedirs(res_path, exist_ok=True)
print(res_path)


In [ ]:
inp_mesh_path = os.path.join(res_path, "inp.obj")
trainer.original_mesh.export(inp_mesh_path)
print(inp_mesh_path)

out_mesh_path = os.path.join(res_path, "ours.obj")
out_mesh.export(out_mesh_path)
print(out_mesh_path)

arap_mesh_path = os.path.join(res_path, "arap.obj")
gtr_mesh.export(arap_mesh_path)
print(arap_mesh_path)

handles_path = os.path.join(res_path, "handles.npy")
np.save(handles_path, handles)
print(handles_path)

targets_path = os.path.join(res_path, "targets.npy")
np.save(targets_path, targets)
print(targets_path)

In [ ]:
print("Gtr")
print("\tarea:", gtr_mesh.area/trainer.original_mesh.area)
print("\tmass:", gtr_mesh.mass/trainer.original_mesh.mass)

print("Ours")
print("\tarea:", out_mesh.area/trainer.original_mesh.area)
print("\tmass:", out_mesh.mass/trainer.original_mesh.mass)


# Draw:
1. Handles: red
2. Targets: blue
3. Shape: silver

In [ ]:
out_mesh_o3d =  o3d.io.read_triangle_mesh(out_mesh_path)
out_mesh_o3d.compute_vertex_normals()

o3d_handles = o3d.geometry.PointCloud()
o3d_handles.points = o3d.utility.Vector3dVector(np.load(handles_path))
o3d_handles.paint_uniform_color([1, 0., 0])

o3d_targets = o3d.geometry.PointCloud()
o3d_targets.points = o3d.utility.Vector3dVector(np.load(targets_path))
o3d_targets.paint_uniform_color([0., 0., 1])


o3d.visualization.draw_geometries([out_mesh_o3d, o3d_handles, o3d_targets], width=1024, height=1024)
# o3d.visualization.draw_geometries([out_mesh_o3d], width=1024, height=1024)


In [ ]:
out_mesh_o3d =  o3d.io.read_triangle_mesh(arap_mesh_path)
out_mesh_o3d.compute_vertex_normals()

o3d_handles = o3d.geometry.PointCloud()
o3d_handles.points = o3d.utility.Vector3dVector(np.load(handles_path))
o3d_handles.paint_uniform_color([1, 0., 0])

o3d_targets = o3d.geometry.PointCloud()
o3d_targets.points = o3d.utility.Vector3dVector(np.load(targets_path))
o3d_targets.paint_uniform_color([0., 0., 1])


o3d.visualization.draw_geometries([out_mesh_o3d, o3d_handles, o3d_targets], width=1024, height=1024)
# o3d.visualization.draw_geometries([out_mesh_o3d], width=1024, height=1024)


In [ ]:
out_mesh_o3d =  o3d.io.read_triangle_mesh(inp_mesh_path)
out_mesh_o3d.compute_vertex_normals()

o3d_handles = o3d.geometry.PointCloud()
o3d_handles.points = o3d.utility.Vector3dVector(np.load(handles_path))
o3d_handles.paint_uniform_color([1, 0., 0])

o3d_targets = o3d.geometry.PointCloud()
o3d_targets.points = o3d.utility.Vector3dVector(np.load(targets_path))
o3d_targets.paint_uniform_color([0., 0., 1])


o3d.visualization.draw_geometries([out_mesh_o3d, o3d_handles, o3d_targets], width=1024, height=1024)


In [ ]:
o3d_handles.paint_uniform_color([1, 0., 0])
o3d.visualization.draw_geometries([out_mesh_o3d, o3d_handles], width=1024, height=1024)